In [1]:
!pip install tensorflow==1.14
!pip install tensorflow_hub
!pip install -U wget
!pip install -U spacy

     |████████████████████████████████| 109.2MB 49kB/s 
     |████████████████████████████████| 491kB 42.8MB/s 
     |████████████████████████████████| 3.2MB 44.9MB/s 
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=7daadebd0e9fb5d8c514a8103ed79822f6647d496d7f99cbfb4487dfb96a309c
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [2]:
# use ELMo as the encoder for the text files
import tensorflow_hub as hub
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
with open('/content/AAPL_0000320193_20171230_item2.txt', 'r') as in_file:
    text = in_file.read()
    sents = nltk.sent_tokenize(text)

In [8]:
embeddings = elmo(sents, signature = "default", as_dict=True)['elmo']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [9]:
embeddings.shape

TensorShape([Dimension(154), Dimension(93), Dimension(1024)])

In [0]:
def elmo_vectors(x):
  # embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [0]:
elmo_train = elmo_vectors(embeddings)

In [0]:
import numpy as np

In [0]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)

In [14]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=edd37bd870f353acfa382bd3837a4a953d482feb07b9a7f440578b949a3696ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-gqecqogf/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [15]:
import wget
from zipfile import ZipFile
import os
import glob
from tqdm import tqdm
import spacy

In [0]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [0]:
import numpy as np
from sklearn.cluster import KMeans
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters)
kmeans = kmeans.fit(elmo_train)

In [0]:
from sklearn.metrics import pairwise_distances_argmin_min
avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, elmo_train)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = ' '.join([sents[closest[idx]] for idx in ordering])

In [22]:
summary

'Europe\n\n\n\n\n\n\n\nEurope net sales increased during the first quarter of 2018 compared to the same quarter in 2017 due primarily to higher net sales of iPhone and Services. In general, the Company believes gross margins will remain under downward pressure due to a variety of factors, including: continued industry-wide global product pricing pressures; increased competition; the Company’s ability to effectively stimulate demand for certain of its products; compressed product life cycles; product transitions; potential increases in the cost of components and outside manufacturing services; the Company’s ability to manage product quality and warranty costs effectively; and a potential shift in the Company’s sales mix towards products with lower gross margins. In addition, the Company sells a variety of third-party Apple-compatible products, including application software and various accessories through its retail and online stores. The Company’s future expected tax rate can be impact